In [308]:
import pretty_midi
import torch
import numpy as np

In [309]:
music_path = './musicDataset/梁祝.mid'
music = pretty_midi.PrettyMIDI(music_path)
# print(music.instruments[0].notes)
notes = music.instruments[0].notes
# print(notes)
timeRun = 0
for i,note in enumerate(notes):
    print(note)
    # if(str(notes[i])!=str(notes[i-1]) and notes[i].start > notes[i-1].end):
    # if(notes[i].pitch!=notes[i-1].pitch and note.start > timeRun):
    #     print(note)
    #     timeRun = note.end

Note(start=0.709091, end=1.409091, pitch=86, velocity=124)
Note(start=0.709091, end=1.409091, pitch=86, velocity=124)
Note(start=0.709091, end=1.409091, pitch=86, velocity=124)
Note(start=1.420455, end=2.120455, pitch=83, velocity=124)
Note(start=1.420455, end=2.120455, pitch=83, velocity=124)
Note(start=1.420455, end=2.120455, pitch=83, velocity=124)
Note(start=2.129545, end=2.829545, pitch=81, velocity=124)
Note(start=2.129545, end=2.829545, pitch=81, velocity=124)
Note(start=2.129545, end=2.829545, pitch=81, velocity=124)
Note(start=2.840909, end=3.190909, pitch=67, velocity=100)
Note(start=3.200000, end=3.550000, pitch=62, velocity=100)
Note(start=2.840909, end=3.550000, pitch=86, velocity=124)
Note(start=3.559091, end=3.909091, pitch=67, velocity=100)
Note(start=3.559091, end=3.909091, pitch=67, velocity=100)
Note(start=3.920455, end=4.270455, pitch=71, velocity=100)
Note(start=3.920455, end=4.270455, pitch=71, velocity=100)
Note(start=2.840909, end=4.270455, pitch=74, velocity=10

In [310]:
t = 0.
roll = list()
unit_time = 0.125
timeRun = 0
# print(notes[1])
# print(notes[2])
# print(str(notes[1])==str(notes[2]))

for i,note in enumerate(notes):
    # if(str(notes[i])!=str(notes[i-1]) and notes[i].start > notes[i-1].end):
    if(note.start >= timeRun):
        timeRun = note.end
        elapsed_time = note.start - t
        if elapsed_time > 0.:
            steps = torch.zeros((int(round(elapsed_time / unit_time)),130))
            steps[range(int(round(elapsed_time / unit_time))),129] += 1
            roll.append(steps)

        n_unites = int(round((note.end - note.start) / unit_time))
        steps = torch.zeros((n_unites,130))
        steps[0,note.pitch] += 1
        steps[range(1,n_unites),128] += 1

        roll.append(steps)
        t = note.end

array = torch.cat(roll,0)

In [311]:
def chord_to_numpy(fpath, unit_time=0.125):
    music = pretty_midi.PrettyMIDI(fpath)
    notes = music.instruments[0].notes
    max_end = 0.
    for note in notes:
        if note.end > max_end:
            max_end = note.end
    chroma = torch.zeros((int(round(max_end / unit_time)), 12))
    for note in notes:
        idx = int(round((note.start / unit_time)))
        n_unit = int(round((note.end - note.start) / unit_time))
        chroma[idx:idx + n_unit, note.pitch % 12] += 1
    return chroma

In [312]:
chroma_array = chord_to_numpy(music_path)

In [313]:
def numpy_to_midi(sample_roll, output='./musicnew.mid'):
    music = pretty_midi.PrettyMIDI()
    piano_program = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
    piano = pretty_midi.Instrument(program=piano_program)
    t = 0
    for i in sample_roll:
        if 'torch' in str(type(i)):
            pitch = int(i.max(0)[1])
        else:
            pitch = int(np.argmax(i))
        if pitch < 128:
            note = pretty_midi.Note(
                velocity=100, pitch=pitch, start=t, end=t + 1 / 8)
            t += 1 / 8
            piano.notes.append(note)
        elif pitch == 128:
            if len(piano.notes) > 0:
                note = piano.notes.pop()
            else:
                p = np.random.randint(60, 72)
                note = pretty_midi.Note(
                    velocity=100, pitch=int(p), start=0, end=t)
            note = pretty_midi.Note(
                velocity=100,
                pitch=note.pitch,
                start=note.start,
                end=note.end + 1 / 8)
            piano.notes.append(note)
            t += 1 / 8
        elif pitch == 129:
            t += 1 / 8
    music.instruments.append(piano)
    music.write(output)

In [314]:
print(str(array))

tensor([[0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]])


In [315]:
def array_to_single(array):
    midi_batch = []
    for i in array:
        num = 0
        for j in i:
            if j == 1:
                midi_batch.append(num)
                num = 0
                break
            num += 1
    return midi_batch

In [316]:
single_array = array_to_single(array)
print(single_array)

[129, 129, 129, 129, 129, 129, 86, 128, 128, 128, 128, 128, 83, 128, 128, 128, 128, 128, 81, 128, 128, 128, 128, 128, 67, 128, 128, 62, 128, 128, 67, 128, 128, 71, 128, 128, 60, 128, 128, 128, 128, 128, 79, 128, 128, 128, 128, 128, 83, 128, 128, 128, 128, 128, 81, 128, 128, 128, 128, 128, 78, 128, 128, 128, 128, 128, 76, 128, 128, 128, 128, 128, 62, 128, 128, 57, 128, 128, 62, 128, 128, 66, 128, 128, 64, 128, 128, 128, 128, 128, 74, 128, 128, 128, 128, 128, 129, 129, 129, 129, 129, 129, 90, 128, 128, 128, 128, 128, 88, 128, 128, 128, 128, 128, 90, 128, 128, 128, 128, 128, 86, 128, 128, 128, 128, 128, 129, 129, 129, 88, 128, 128, 85, 128, 128, 128, 128, 128, 83, 128, 128, 128, 128, 128, 81, 128, 128, 83, 128, 128, 85, 128, 128, 83, 128, 128, 86, 128, 128, 128, 128, 128, 129, 129, 129, 83, 128, 128, 81, 128, 128, 83, 128, 128, 86, 128, 128, 88, 128, 128, 83, 128, 128, 85, 128, 128, 83, 128, 128, 81, 128, 128, 129, 129, 129, 129, 129, 129, 83, 128, 128, 81, 128, 128, 79, 128, 128, 128, 12

In [317]:
def single_to_array(single):
    midi_array = []
    for i in single:
        z = [0] * 130
        z[int(i)] = 1
        midi_array.append(z)
    return midi_array

In [318]:
new_array = torch.LongTensor(single_to_array(single_array))
print(new_array)

tensor([[0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        ...,
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 1, 0]])


In [319]:
numpy_to_midi(new_array)

In [321]:
from pretty_midi import instrument
other_array = music.instruments[0].get_piano_roll()
other_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])